In [1]:
%load_ext autoreload
%autoreload 2

import jax 
import jax.numpy as jnp 

from transformers import (
    FlaxAutoModel, 
    AutoTokenizer,
    FlaxGemmaForCausalLM,
    pipeline
)

from dotenv import load_dotenv
load_dotenv()

/Users/ericliu/research/curious/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [16]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

In [20]:
model = FlaxAutoModel.from_pretrained("openai-community/gpt2")

In [17]:
inputs = tokenizer("What's 1+1 = 2", return_tensors="jax")

In [21]:
outputs = model(**inputs)

In [22]:
last_hidden_states = outputs.last_hidden_state

In [26]:
print(type(outputs))

<class 'transformers.modeling_flax_outputs.FlaxBaseModelOutputWithPastAndCrossAttentions'>


In [28]:
last_hidden_states.argmax(axis=-1)

Array([[496, 496, 496, 496, 496, 496, 496]], dtype=int32)